In [1]:
import sys
import os

In [2]:
import pandas as pd

In [3]:
from tqdm import tqdm

In [4]:
from azure.cosmos import CosmosClient, PartitionKey

In [5]:
sys.path.append('..')

In [6]:
import config

In [7]:
df = pd.read_csv("../data/PS_20174392719_1491204439457_log.csv")

In [8]:
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


Andmete kirjeldus kaggle-i lehelt:
```
1,PAYMENT,1060.31,C429214117,1089.0,28.69,M1591654462,0.0,0.0,0,0
```
  * step - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).
  * type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.
  * amount - amount of the transaction in local currency.
  * nameOrig - customer who started the transaction
  * oldbalanceOrg - initial balance before the transaction
  * newbalanceOrig - new balance after the transaction
  * nameDest - customer who is the recipient of the transaction
  * oldbalanceDest - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).
  * newbalanceDest - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).
  * isFraud - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.
  * isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.



## Sisestame osa andmeid CosmosDB-sse katsetuseks

In [9]:
df.index.names = ['id']

In [10]:
df = df.reset_index().set_index('step', drop=False)

In [11]:
df

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
step,,,,,,,,,,,,
1,0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
1,2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
1,3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
1,4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
743,6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
743,6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
743,6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0


In [12]:
df_insert = df.loc[1:3]

In [13]:
df_insert

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
step,,,,,,,,,,,,
1,0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
1,2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
1,3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
1,4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3,4269,3,PAYMENT,22421.09,C516570791,121670.0,99248.91,M620471432,0.0,0.0,0,0
3,4270,3,PAYMENT,15323.44,C1940311161,9083.0,0.00,M1515016092,0.0,0.0,0,0
3,4271,3,PAYMENT,26862.79,C356347252,26044.0,0.00,M1822115007,0.0,0.0,0,0


In [14]:
df_insert.head(3).to_dict('records')

[{'id': 0,
  'step': 1,
  'type': 'PAYMENT',
  'amount': 9839.64,
  'nameOrig': 'C1231006815',
  'oldbalanceOrg': 170136.0,
  'newbalanceOrig': 160296.36,
  'nameDest': 'M1979787155',
  'oldbalanceDest': 0.0,
  'newbalanceDest': 0.0,
  'isFraud': 0,
  'isFlaggedFraud': 0},
 {'id': 1,
  'step': 1,
  'type': 'PAYMENT',
  'amount': 1864.28,
  'nameOrig': 'C1666544295',
  'oldbalanceOrg': 21249.0,
  'newbalanceOrig': 19384.72,
  'nameDest': 'M2044282225',
  'oldbalanceDest': 0.0,
  'newbalanceDest': 0.0,
  'isFraud': 0,
  'isFlaggedFraud': 0},
 {'id': 2,
  'step': 1,
  'type': 'TRANSFER',
  'amount': 181.0,
  'nameOrig': 'C1305486145',
  'oldbalanceOrg': 181.0,
  'newbalanceOrig': 0.0,
  'nameDest': 'C553264065',
  'oldbalanceDest': 0.0,
  'newbalanceDest': 0.0,
  'isFraud': 1,
  'isFlaggedFraud': 0}]

In [15]:
group_per_step_and_id = df_insert.set_index('step').groupby(['step','id'])

Python SDK docks - https://docs.microsoft.com/en-us/azure/cosmos-db/create-sql-api-python

In [16]:
endpoint = config.config['cosmosdb_config']['COSMOSDB_HOST']
key = config.config['cosmosdb_config']['COSMOSDB_KEY']
database_name = "rawdata"
container_name = "payments"

In [17]:
client = CosmosClient(endpoint, key)

In [18]:
database = client.create_database_if_not_exists(id=database_name)

In [19]:
# resetting the container

containers = database.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database.delete_container(container_name)
    print('Container dropped')

In [20]:
container = database.create_container_if_not_exists(
    id=container_name, 
    partition_key=PartitionKey(path="/step"),
    offer_throughput=400
)

print(f"Container {container} created ")

Container <ContainerProxy [dbs/rawdata/colls/payments]> created 


Sisestame kolm esimest päeva andmebaasi

In [21]:
for (step, id), group in tqdm(group_per_step_and_id):
    
    payload=group.to_dict('records')[0]
    payload['step']=str(step)
    payload['id']=str(id)
    
    #print(payload)
    
    container.create_item(body=payload)        

100%|██████████| 4274/4274 [04:14<00:00, 16.80it/s]
